In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import plotly.graph_objects as go

import random
import pandas as pd
import numpy as np

!pip install torchinfo
from torchinfo import summary

In [10]:
# Random Seed 고정 (학습 반복 시행 시에도 동일한 결과가 나오도록)

seed = 20250309

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

**1. 데이터셋 로딩 및 데이터 분석**

In [11]:
# 데이터셋 로딩

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True,
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False,
                                            transform=transform,
                                            download=True)


Files already downloaded and verified
Files already downloaded and verified


In [12]:
# 원본 Class (총 10개) 의 이미지를 대상으로 테스트하면서 정확도가 어느 정도 나오려면
# 학습 시간이 너무 오래 걸릴 정도로 많은 양의 데이터를 학습해야 함
# -> 실험 시간 절약을 위해, 4개 Class 의 데이터로 필터링

# 또한 시간 절약을 위해, 학습 데이터에서 랜덤하게 일부 샘플만 추출

from torch.utils.data import Subset

NUM_TRAIN_SAMPLES = 9000
filter_labels = {0, 1, 2, 3}

# train data
train_filter_indices = [i for i, label in enumerate(train_dataset.targets) if label in filter_labels]
train_subset_indices = random.sample(train_filter_indices, NUM_TRAIN_SAMPLES)
train_subset = Subset(train_dataset, train_subset_indices)

# test data
test_subset_indices = [i for i, label in enumerate(test_dataset.targets) if label in filter_labels]
test_subset = Subset(test_dataset, test_subset_indices)

In [13]:
# Data Loader 생성

from torch.utils.data import Subset, DataLoader

BATCH_SIZE = 32

train_loader = DataLoader(train_subset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

# 테스트 데이터셋은 학습 대상이 아니므로 그대로 이용
test_loader = DataLoader(test_subset,
                         batch_size=BATCH_SIZE,
                         shuffle=False)

In [14]:
# 클래스 불균형 분석

# 학습 데이터
train_labels = torch.tensor([train_subset.dataset.targets[i] for i in train_subset_indices])
train_class_counts = torch.bincount(train_labels)
print(train_class_counts)

NUM_CLASSES_ORIGINAL = len(train_class_counts)
NUM_CLASSES = len(filter_labels)

tensor([2275, 2217, 2235, 2273])


In [15]:
train_class_percentage = np.array(train_class_counts) * 100.0 / sum(train_class_counts)

train_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES_ORIGINAL)),
                                'count': train_class_counts,
                                'percentage (%)': train_class_percentage})

train_y_distrib

,class,count,percentage (%)
0,0,2275,25.277778
1,1,2217,24.633334
2,2,2235,24.833334
3,3,2273,25.255556


In [16]:
# 테스트 데이터
test_labels = torch.tensor([test_subset.dataset.targets[i] for i in test_subset_indices])
test_class_counts = torch.bincount(torch.tensor(test_labels))
print(test_class_counts)

tensor([1000, 1000, 1000, 1000])


<ipython-input-16-a6116744976a>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_class_counts = torch.bincount(torch.tensor(test_labels))


In [17]:
test_class_percentage = np.array(test_class_counts) * 100.0 / sum(test_class_counts)

test_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                               'count': test_class_counts,
                               'percentage (%)': test_class_percentage})

test_y_distrib

,class,count,percentage (%)
0,0,1000,25.000001
1,1,1000,25.000001
2,2,1000,25.000001
3,3,1000,25.000001


In [18]:
# CNN 모델 정의

class CNN(nn.Module):

    def __init__(self, padding_options):
        super(CNN, self).__init__()
        self.padding_options = padding_options

        padding_conv1 = padding_options[0]
        padding_conv2 = padding_options[1]
        padding_conv4 = padding_options[2]

        # Conv
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, padding=2, padding_mode=padding_conv1),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, padding=2, padding_mode=padding_conv2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, padding=2, padding_mode=padding_conv4),
            nn.ReLU()
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3),
            nn.ReLU()
        )

        # Fully Connected
        self.fc1 = nn.Sequential(
            nn.Linear(256 * 5 * 5, 256),
            nn.Sigmoid()
        )
        self.fc_final = nn.Sequential(
            nn.Linear(256, 10),
            nn.Softmax()  # Classification Task 의 Output Layer 이므로 Softmax 고정
        )

    def forward(self, x):

        # Conv
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool2(x)
        x = self.conv4(x)
        x = self.conv5(x)

        x = x.view(-1, 256 * 5 * 5)

        # Fully Connected
        x = self.fc1(x)
        x = self.fc_final(x)

        return x

In [19]:
# 모델 구조 출력

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(padding_options=['zeros', 'reflect', 'replicate']).to(device)

print(summary(model, input_size=(BATCH_SIZE, 3, 32, 32)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [32, 10]                  --
├─Sequential: 1-1                        [32, 32, 32, 32]          --
│    └─Conv2d: 2-1                       [32, 32, 32, 32]          2,432
│    └─ReLU: 2-2                         [32, 32, 32, 32]          --
├─MaxPool2d: 1-2                         [32, 32, 16, 16]          --
├─Sequential: 1-3                        [32, 64, 16, 16]          --
│    └─Conv2d: 2-3                       [32, 64, 16, 16]          51,264
│    └─ReLU: 2-4                         [32, 64, 16, 16]          --
├─Sequential: 1-4                        [32, 64, 14, 14]          --
│    └─Conv2d: 2-5                       [32, 64, 14, 14]          36,928
│    └─ReLU: 2-6                         [32, 64, 14, 14]          --
├─MaxPool2d: 1-5                         [32, 64, 7, 7]            --
├─Sequential: 1-6                        [32, 128, 7, 7]           --
│   

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1844: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return inner()


**3. 데이터셋 분리**

* Train Data -> Train Data + Valid Data (epoch) + Valid Data (하이퍼파라미터 최적화)

In [20]:
# 데이터셋 분리

from torch.utils.data import random_split

# 샘플 수
num_train = 2000
num_valid_epoch = 2000
num_valid_hpo = 5000

assert NUM_TRAIN_SAMPLES == num_train + num_valid_epoch + num_valid_hpo

# 데이터셋 분리
train_dataset, valid_epoch_dataset, valid_hpo_dataset =\
    random_split(train_subset, [num_train, num_valid_epoch, num_valid_hpo])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_epoch_loader = DataLoader(valid_epoch_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_hpo_loader = DataLoader(valid_hpo_dataset, batch_size=BATCH_SIZE, shuffle=False)

**4. 하이퍼파라미터 최적화 학습 실시 함수**

* 하이퍼파라미터 최적화 라이브러리는 Optuna 사용
* 하이퍼파라미터 탐색 200 회 실시
* 하이퍼파라미터 목록
  * 레이어 별 Padding 의 종류
    * 레이어 : Conv1, Conv2, Conv4
    * Padding 의 종류 : zeros, reflect, replicate, circular
  * Learning Rate
    * 0.00003 ~ 0.001 (= 3e-5 ~ 1e-3) 범위


In [21]:
MAX_EPOCHS = 65536
TRIAL_COUNT = 200           # HPO trial count
EARLY_STOPPING_ROUNDS = 10  # Early Stopping epoch count

In [22]:
from sklearn.metrics import accuracy_score
from copy import deepcopy

In [23]:
# Optuna 설정

!pip install optuna
import optuna
import logging

optuna.logging.set_verbosity(logging.WARNING)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.2 MB/s eta 0:00:00


In [24]:
# 모델 학습 실시

# args :
# - model           : 학습할 모델
# - train_loader    : Training Data Loader
# - train_loss_list : 각 epoch 에서의 train loss 기록

# returns :
# - train_loss : 모델의 Train Loss

def run_train(model, train_loader, train_loss_list):
    model.train()
    train_loss = 0.0
    cnt = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # train 실시
        model.optimizer.zero_grad()
        outputs = model(images)

        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        model.optimizer.step()

        train_loss += loss.item()
        cnt += 1

    train_loss_list.append(train_loss / len(train_loader))
    return train_loss_list[-1]

In [25]:
# 모델 validation 실시

# args :
# - model        : validation 할 모델
# - valid_loader : Validation Data Loader
# - during_train : 모델 학습 중이면 True, 그렇지 않으면 False

# returns :
# - val_accuracy : 모델의 validation 정확도
# - val_loss     : 모델의 validation loss

def run_validation(model, valid_loader, during_train=True):
    model.eval()
    correct, total = 0, 0
    val_loss_sum = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss_batch = nn.CrossEntropyLoss(reduction='sum')(outputs, labels)
            val_loss_sum += val_loss_batch

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Accuracy 계산
        val_accuracy = correct / total
        val_loss = val_loss_sum / total

    return val_accuracy, val_loss

In [26]:
# 모델 학습 및 validation 전체 프로세스

# args :
# - model              : 학습할 모델
# - train_loader       : Training Data Loader
# - valid_epoch_loader : 각 epoch 마다 validation 할 Valid Data Loader
# - valid_hpo_loader   : 최종적으로 해당 하이퍼파라미터 조합에 대한 Valid Data Loader
# - verbose            : 학습 중 프로세스 출력 여부

# returns :
# - final_acc        : 해당 하이퍼파라미터 조합에 대한 최종 Accuracy (valid accuracy 가 가장 높았던 epoch 의 모델로 측정)
# - best_epoch_model : valid accuracy 가 가장 높았던 epoch 에서 생성된 모델
# - epochs           : 해당 학습의 총 epoch count

def run_model_common(model, train_loader, valid_epoch_loader, valid_hpo_loader,
                     verbose=False):

    train_loss_list = []        # train loss
    valid_acc_list = []         # valid accuracy
    valid_loss_list = []        # valid loss

    max_valid_acc = 0.0         # max validation accuracy
    min_valid_loss = None       # min validation loss

    best_valid_acc_epoch = -1   # valid accuracy 가 가장 높았던 epoch
    best_valid_loss_epoch = -1  # valid loss 가 가장 낮았던 epoch
    best_epoch_model = None     # valid accuracy 가 가장 높았던 epoch 의 모델

    # 1. 학습 실시
    for epoch in range(MAX_EPOCHS):

        # 1-1. train model
        train_loss = run_train(model, train_loader, train_loss_list)

        # 1-2. validate model (with EPOCH VALID SET)
        epoch_acc, val_loss = run_validation(model, valid_epoch_loader)
        valid_acc_list.append(epoch_acc)
        valid_loss_list.append(val_loss)

        # 1-3. Early Stopping 처리 (overfitting 방지)
        if epoch_acc > max_valid_acc:
            max_valid_acc = epoch_acc
            best_valid_acc_epoch = epoch

            best_epoch_model = CNN(padding_options=model.padding_options).to(device)
            best_epoch_model.load_state_dict(model.state_dict())

            if verbose:
                print('best model updated')

        # Early Stopping 학습 종료
        if epoch - best_valid_acc_epoch >= EARLY_STOPPING_ROUNDS:
            epochs = epoch
            break

        # 1-4. 결과 출력
        if verbose:
            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {val_loss:.4f}, Valid Accuracy: {epoch_acc:.4f}")

    # check best-epoch model correctly loaded
    checked_acc, _ = run_validation(best_epoch_model,
                                    valid_epoch_loader,
                                    during_train=False)

    if verbose:
        print(f"Best Epoch: {best_valid_acc_epoch}, Best Valid Acc: {max_valid_acc}")
        print(f"Valid Acc (with Epoch valid set) on Loaded Best Model: {checked_acc}")

    assert abs(max_valid_acc - checked_acc) < 1e-8

    # 2. validate best-epoch model (with HPO VALID SET)
    final_acc, _ = run_validation(best_epoch_model,
                                  valid_hpo_loader,
                                  during_train=False)

    if verbose:
        print(f"Final Acc (with HPO valid set) on Loaded Best Model: {final_acc}")

    return final_acc, best_epoch_model, epochs

In [27]:
print(device)

cuda


**4-1. 실험 실시**

In [28]:
hpo_best_acc = 0              # 모든 Hyper-param 조합의 HPO Valid set 정확도 중 가장 높은 것
best_hyperparam_set = None    # HPO Valid set 정확도가 가장 높은 Hyper-param 조합
best_hyperparam_model = None  # best_hyperparam_set 의 Hyper-param 조합으로 학습된 모델

In [29]:
trial_count = 0   # 1st ~ 20th trial 에만 학습 중 정보 출력
epoch_count = []  # 각 trial 의 epoch 횟수 리스트

def objective(trial):
    global hpo_best_acc, best_hyperparam_set, best_hyperparam_model, trial_count, epoch_count

    # hyper-params
    padding_modes = ['zeros', 'reflect', 'replicate', 'circular']

    params = {
        'padding_mode_conv1': trial.suggest_categorical('padding_mode_conv1', padding_modes),
        'padding_mode_conv2': trial.suggest_categorical('padding_mode_conv2', padding_modes),
        'padding_mode_conv4': trial.suggest_categorical('padding_mode_conv4', padding_modes),
        'learning_rate': trial.suggest_float('learning_rate', 0.00003, 0.001, log=True)
    }

    # define and run model
    pad_conv1 = params['padding_mode_conv1']
    pad_conv2 = params['padding_mode_conv2']
    pad_conv4 = params['padding_mode_conv4']

    model = CNN(padding_options=[pad_conv1, pad_conv2, pad_conv4]).to(device)

    model.optimizer = torch.optim.AdamW(model.parameters(),
                                        lr=params['learning_rate'])

    final_acc, best_epoch_model, epochs = run_model_common(model,
                                                           train_loader,
                                                           valid_epoch_loader,
                                                           valid_hpo_loader,
                                                           verbose=(trial_count < 20))

    trial_count += 1
    epoch_count.append(epochs)

    # global best model 갱신
    if final_acc > hpo_best_acc:
        hpo_best_acc = final_acc
        best_hyperparam_set = params

        best_hyperparam_model = CNN(padding_options=[pad_conv1, pad_conv2, pad_conv4]).to(device)
        best_hyperparam_model.load_state_dict(best_epoch_model.state_dict())

        print(f'best_hyperparam_model updated with Accuracy={hpo_best_acc:.4f}')

    print(f"Trial: {trial_count}, Params: {params}, Accuracy: {final_acc:.4f}")
    return final_acc

In [30]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=TRIAL_COUNT)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 2.2159, Valid Loss: 2.1675, Valid Accuracy: 0.2590
best model updated
Epoch 2, Train Loss: 2.1547, Valid Loss: 2.1393, Valid Accuracy: 0.3270
best model updated
Epoch 3, Train Loss: 2.1230, Valid Loss: 2.1093, Valid Accuracy: 0.4065
best model updated
Epoch 4, Train Loss: 2.0791, Valid Loss: 2.0442, Valid Accuracy: 0.4895
best model updated
Epoch 5, Train Loss: 1.9927, Valid Loss: 1.9731, Valid Accuracy: 0.5515
Epoch 6, Train Loss: 1.9567, Valid Loss: 1.9711, Valid Accuracy: 0.5375
best model updated
Epoch 7, Train Loss: 1.9268, Valid Loss: 1.9452, Valid Accuracy: 0.5590
best model updated
Epoch 8, Train Loss: 1.9046, Valid Loss: 1.9205, Valid Accuracy: 0.5785
best model updated
Epoch 9, Train Loss: 1.8888, Valid Loss: 1.9164, Valid Accuracy: 0.5790
best model updated
Epoch 10, Train Loss: 1.8881, Valid Loss: 1.8962, Valid Accuracy: 0.5940
best model updated
Epoch 11, Train Loss: 1.8657, Valid Loss: 1.8767, Valid Accuracy: 0.6135
Epoch 12, Train 

In [31]:
# Test Dataset 성능 평가

print(f'best hyper-param: {best_hyperparam_set}, best acc: {hpo_best_acc}')

best hyper-param: {'padding_mode_conv1': 'reflect', 'padding_mode_conv2': 'circular', 'padding_mode_conv4': 'replicate', 'learning_rate': 0.0001682827416120558}, best acc: 0.7378


In [32]:
# best_hyperparam_model 이 정상적으로 load 되었는지 최종 확인

checked_hpo_acc, _ = run_validation(best_hyperparam_model,
                                    valid_hpo_loader,
                                    during_train=False)

print(f"Valid Acc (with HPO valid set) on Best Hyper-param Model: {checked_hpo_acc}")

assert abs(hpo_best_acc - checked_hpo_acc) < 1e-8

Valid Acc (with HPO valid set) on Best Hyper-param Model: 0.7378


In [33]:
# 테스트셋에 대한 최종 정확도

hpo_final_acc, _ = run_validation(best_hyperparam_model,
                                  test_loader,
                                  during_train=False)

print(f'Final HPO Acc (with test set) : {hpo_final_acc}')

Final HPO Acc (with test set) : 0.74475


**5. HPO 성능 결과 확인**

In [34]:
from optuna.visualization import plot_optimization_history

In [91]:
# HPO 추이

fig = plot_optimization_history(study)
fig.update_layout(width=1000,
                  height=650,
                  yaxis_title='Accuracy (HPO valid set)')
fig.show()

In [92]:
fig.update_layout(yaxis=dict(range=[0.64, 0.74]))
fig.show()

**6. 각 Hyperparameter 값에 따른 성능 분포 확인**

In [93]:
# trial DataFrame 가져오기

trials_df = study.trials_dataframe()

In [94]:
trials_df

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_padding_mode_conv1,params_padding_mode_conv2,params_padding_mode_conv4,state
0,0,0.6692,2025-03-08 23:40:50.872626,2025-03-08 23:42:09.343831,0 days 00:01:18.471205,0.000053,circular,zeros,reflect,COMPLETE
1,1,0.6868,2025-03-08 23:42:09.344032,2025-03-08 23:44:42.682835,0 days 00:02:33.338803,0.000032,replicate,reflect,reflect,COMPLETE
2,2,0.2412,2025-03-08 23:44:42.683058,2025-03-08 23:45:00.623016,0 days 00:00:17.939958,0.000438,reflect,replicate,zeros,COMPLETE
3,3,0.2412,2025-03-08 23:45:00.623181,2025-03-08 23:45:18.014536,0 days 00:00:17.391355,0.000788,circular,replicate,zeros,COMPLETE
4,4,0.2412,2025-03-08 23:45:18.014727,2025-03-08 23:45:35.977501,0 days 00:00:17.962774,0.000158,zeros,zeros,zeros,COMPLETE
...,...,...,...,...,...,...,...,...,...,...
195,195,0.7114,2025-03-09 03:32:56.370133,2025-03-09 03:34:02.054356,0 days 00:01:05.684223,0.000148,reflect,zeros,circular,COMPLETE
196,196,0.2412,2025-03-09 03:34:02.054547,2025-03-09 03:34:19.637959,0 days 00:00:17.583412,0.000228,reflect,zeros,circular,COMPLETE
197,197,0.7032,2025-03-09 03:34:19.638146,2025-03-09 03:36:11.549924,0 days 00:01:51.911778,0.000139,reflect,circular,zeros,COMPLETE
198,198,0.2412,2025-03-09 03:36:11.550147,2025-03-09 03:36:28.704007,0 days 00:00:17.153860,0.000280,reflect,circular,circular,COMPLETE


In [95]:
# 차트 표시 순서를 일정하게

category_orders = ['zeros', 'reflect', 'replicate', 'circular']

In [96]:
# Early Stopping Type 별, Learning Rate 에 따른 Accuracy 분포

import plotly.express as px

fig = px.scatter(trials_df,
                 x="params_learning_rate",
                 y="value",
                 color="params_padding_mode_conv1",
                 category_orders={'params_padding_mode_conv1': category_orders},
                 title="Accuracy Distribution by Conv.1 Padding Type & Learning Rate")

fig.update_layout(width=1000, height=600,
                  xaxis_title='Padding (Conv. 1 Layer)',
                  yaxis_title='Accuracy')

fig.show()

In [97]:
fig.update_layout(xaxis=dict(range=[0, 0.0005]),
                  yaxis=dict(range=[0.64, 0.74]))
fig.show()

In [98]:
fig = px.scatter(trials_df,
                 x="params_learning_rate",
                 y="value",
                 color="params_padding_mode_conv2",
                 category_orders={'params_padding_mode_conv2': category_orders},
                 title="Accuracy Distribution by Conv.2 Padding Type & Learning Rate")

fig.update_layout(width=1000, height=600,
                  xaxis_title='Padding (Conv. 2 Layer)',
                  yaxis_title='Accuracy')

fig.show()

In [99]:
fig.update_layout(xaxis=dict(range=[0, 0.0005]),
                  yaxis=dict(range=[0.64, 0.74]))
fig.show()

In [100]:
fig = px.scatter(trials_df,
                 x="params_learning_rate",
                 y="value",
                 color="params_padding_mode_conv4",
                 category_orders={'params_padding_mode_conv4': category_orders},
                 title="Accuracy Distribution by Conv.4 Padding Type & Learning Rate")

fig.update_layout(width=1000, height=600,
                  xaxis_title='Padding (Conv. 4 Layer)',
                  yaxis_title='Accuracy')

fig.show()

In [101]:
fig.update_layout(xaxis=dict(range=[0, 0.0005]),
                  yaxis=dict(range=[0.64, 0.74]))
fig.show()